In [ ]:
!pip install fastapi uvicorn python-multipart scikit-learn pandas numpy pyngrok joblib -q

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
import os

# Gerando dados sintéticos semelhantes ao Boston Housing
np.random.seed(42)
n_samples = 10000

# Features típicas do Boston
data = {
    'CRIM': np.random.lognormal(0, 1, n_samples),        # taxa de criminalidade
    'ZN': np.random.uniform(0, 100, n_samples),          # proporção de terrenos residenciais
    'INDUS': np.random.uniform(0, 30, n_samples),       # proporção de acres comerciais
    'CHAS': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),  # perto do rio Charles?
    'NOX': np.random.uniform(0.3, 0.9, n_samples),       # concentração de óxidos nítricos
    'RM': np.random.normal(6.5, 1, n_samples),           # média de quartos por habitação
    'AGE': np.random.uniform(1, 100, n_samples),         # idade da casa
    'DIS': np.random.lognormal(1, 0.8, n_samples),       # distância dos centros de emprego
    'RAD': np.random.choice(range(1,25), n_samples),     # índice de acessibilidade a rodovias
    'TAX': np.random.uniform(200, 800, n_samples),      # taxa de imposto predial
    'PTRATIO': np.random.uniform(12, 22, n_samples),    # relação aluno-professor
    'B': np.random.uniform(0, 400, n_samples),          # proporção de população negra
    'LSTAT': np.random.uniform(1, 40, n_samples),       # % de população de baixa renda
}

df = pd.DataFrame(data)

# Target: preço da casa (em milhares de dólares) - relação não-linear realista
df['MEDV'] = (
    10 +
    4 * df['RM'] -
    0.2 * df['LSTAT'] +
    3 * df['CHAS'] -
    2 * np.log1p(df['CRIM']) +
    1.5 * (df['ZN']/10) -
    0.05 * df['AGE'] +
    np.random.normal(0, 3, n_samples)
)

# Garantir valores positivos
df['MEDV'] = df['MEDV'].clip(lower=5)

print(df.head())
print(df.shape)

       CRIM         ZN      INDUS  CHAS       NOX        RM        AGE  \
0  1.643313  32.125517   5.145295     0  0.454104  7.104467  41.327874   
1  0.870868  10.301919  10.404067     0  0.743468  7.094179   4.393514   
2  1.911118   7.180755  29.103053     0  0.550236  6.815981  18.200516   
3  4.586099   9.427598  23.423635     0  0.381747  7.561121  48.496981   
4  0.791240  58.286936   8.477725     0  0.453093  7.299532  40.256329   

        DIS  RAD         TAX    PTRATIO           B      LSTAT       MEDV  
0  1.624952    3  591.881824  18.350837  246.061278   1.540394  32.052305  
1  1.634267    9  450.832660  20.453184  143.551350  26.891644  32.320710  
2  4.869825    8  430.994658  14.219245  314.965278  28.710923  32.078273  
3  7.473832    8  466.970063  18.150023  318.622900  37.544237  33.212119  
4  1.107771    1  326.790767  21.933302  194.447399   4.328320  45.262780  
(10000, 14)


In [ ]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Salvar o modelo
os.makedirs('/content/model', exist_ok=True)
joblib.dump(model, '/content/model/boston_model.pkl')

# Teste rápido
print("R² no teste:", model.score(X_test, y_test))

R² no teste: 0.7825445643027493


In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel, Field
import joblib
import pandas as pd
from typing import Literal

# Carregar o modelo
model = joblib.load('/content/model/boston_model.pkl')

app = FastAPI(
    title="API de Previsão de Preço de Casas - Boston",
    description="Modelo de regressão com RandomForest",
    version="1.0.0"
)

# Classe Pydantic para validar o payload de entrada
class HouseFeatures(BaseModel):
    CRIM: float = Field(..., example=0.00632, description="Taxa de criminalidade per capita")
    ZN: float = Field(..., example=18.0, description="Proporção de terreno residencial")
    INDUS: float = Field(..., example=2.31, description="Proporção de acres comerciais")
    CHAS: int = Field(..., example=0, description="1 se beira o rio Charles, 0 caso contrário", ge=0, le=1)
    NOX: float = Field(..., example=0.538, description="Concentração de óxidos nítricos")
    RM: float = Field(..., example=6.575, description="Média de quartos por habitação")
    AGE: float = Field(..., example=65.2, description="Proporção de unidades construídas antes de 1940")
    DIS: float = Field(..., example=4.09, description="Distância ponderada aos centros de emprego")
    RAD: int = Field(..., example=1, description="Índice de acessibilidade a rodovias")
    TAX: float = Field(..., example=296.0, description="Taxa de imposto predial")
    PTRATIO: float = Field(..., example=15.3, description="Relação aluno-professor")
    B: float = Field(..., example=396.9, description="Proporção da população negra")
    LSTAT: float = Field(..., example=4.98, description="% da população de baixa renda")

@app.get("/")
async def root():
    return {"message": "API de previsão de preço de casas em Boston. Acesse /docs para testar!"}

@app.post("/predict")
async def predict_price(house: HouseFeatures):
    # Converter para DataFrame (exatamente como foi treinado)
    input_data = pd.DataFrame([house.dict()])

    # Previsão
    prediction = model.predict(input_data)[0]

    return {
        "predicted_price_thousands_usd": round(float(prediction), 2),
        "currency": "USD",
        "note": "Preço estimado em milhares de dólares"
    }

Writing main.py


In [ ]:
# Mata qualquer processo que esteja usando a porta 8000
!fuser -k 8000/tcp
!pkill uvicorn
!pkill ngrok
!pkill cloudflared

In [ ]:
import uvicorn
import threading

def run():
    uvicorn.run("main:app", host="0.0.0.0", port=8000, log_level="info")

# Inicia o servidor limpo
threading.Thread(target=run, daemon=True).start()
print("API rodando! Aguarde os links abaixo...")

API rodando! Aguarde os links abaixo...


In [ ]:
from google.colab import output

# Abre em nova aba (recomendado)
output.serve_kernel_port_as_window(port=8000)

# Abre embutido no notebook (opcional)
output.serve_kernel_port_as_iframe(port=8000, height=800)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import output
output.serve_kernel_port_as_iframe(port=8000, height=800, path='/docs')

<IPython.core.display.Javascript object>

**BATCH PREDICTION** + **API KEY**

In [ ]:
%%writefile main.py
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
from typing import List
import joblib
import pandas as pd

# Carrega o modelo
model = joblib.load('/content/model/boston_model.pkl')

app = FastAPI(
    title="API Boston Housing - Produção",
    description="Batch prediction + API Key (produção)",
    version="3.0.0"
)

# MUDE ESSA CHAVE PARA UMA FORTE! (depois te mostro como gerar uma aleatória)
API_KEY = "boston_2025_secret_8f9d2a1c9e7b3f6d5a4c8e2f1d0b9a8e7c6d5f4"

class HouseFeatures(BaseModel):
    CRIM: float = Field(..., example=0.00632)
    ZN: float = Field(..., example=18.0)
    INDUS: float = Field(..., example=2.31)
    CHAS: int = Field(..., example=0, ge=0, le=1)
    NOX: float = Field(..., example=0.538)
    RM: float = Field(..., example=6.575)
    AGE: float = Field(..., example=65.2)
    DIS: float = Field(..., example=4.09)
    RAD: int = Field(..., example=1)
    TAX: float = Field(..., example=296.0)
    PTRATIO: float = Field(..., example=15.3)
    B: float = Field(..., example=396.9)
    LSTAT: float = Field(..., example=4.98)

@app.get("/")
async def root():
    return {"message": "API protegida com API Key + batch. Acesse /docs"}

@app.post("/predict")
async def predict_price(
    houses: List[HouseFeatures],
    x_api_key: str = Header(..., alias="X-API-Key")
):
    if x_api_key != API_KEY:
        raise HTTPException(status_code=401, detail="API Key inválida")

    if len(houses) == 0:
        raise HTTPException(status_code=400, detail="Lista vazia")

    df = pd.DataFrame([h.dict() for h in houses])
    predictions = model.predict(df)

    return {
        "total_predictions": len(predictions),
        "prices_thousands_usd": [round(float(p), 2) for p in predictions],
        "model": "RandomForestRegressor",
        "status": "success"
    }

Overwriting main.py


In [ ]:
# Mata o antigo
!fuser -k 8000/tcp

# Roda o novo
import uvicorn, threading
threading.Thread(target=lambda: uvicorn.run("main:app", host="0.0.0.0", port=8000), daemon=True).start()

# Abre direto no Swagger (nova aba automática)
from google.colab import output
output.serve_kernel_port_as_window(port=8000, path='/docs')

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [ ]:
from google.colab import output
output.serve_kernel_port_as_iframe(port=8000, height=1000, path='/docs')

**DEPLOY NO RAILWAY**

In [ ]:
# ========================================
# VERSÃO IMORTAL — FUNCIONA 1000000% NO COLAB
# ========================================
!rm -rf boston-housing-api
!mkdir -p boston-housing-api
%cd boston-housing-api

# requirements.txt
!echo -e "fastapi\nuvicorn[standard]\nscikit-learn\npandas\njoblib\ngunicorn" > requirements.txt

# Procfile
!echo "web: uvicorn main:app --host 0.0.0.0 --port \${PORT:-5000}" > Procfile

# Copia modelo e código
!cp /content/model/boston_model.pkl .
!cp /content/main.py .

# README.md (só texto puro, sem acento nem nada)
!echo -e "# Boston Housing Price Prediction API\n\nAPI com FastAPI\n\n- Batch prediction\n- API Key\n- Modelo RandomForest\n\nPOST /predict\nHeader: X-API-Key: boston_2025_secret_8f9d2a1c9e7b3f6d5a4c8e2f1d0b9a8e7c6d5f4\n\nDocs: /docs" > README.md

!ls -la

/content/boston-housing-api/boston-housing-api
cp: cannot stat '/content/main.py': No such file or directory
total 142324
drwxr-xr-x 2 root root      4096 Nov 24 01:11 .
drwxr-xr-x 4 root root      4096 Nov 24 01:11 ..
-rw-r--r-- 1 root root 145717329 Nov 24 01:11 boston_model.pkl
-rw-r--r-- 1 root root        58 Nov 24 01:11 Procfile
-rw-r--r-- 1 root root       213 Nov 24 01:11 README.md
-rw-r--r-- 1 root root        62 Nov 24 01:11 requirements.txt


In [ ]:
# GERA O MAIN.PY FINAL COM MODELO EMBUTIDO EM BASE64 (NUNCA DÁ ERRO)
%%writefile main_final.py
import joblib
import base64
import io
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
import numpy as np

app = FastAPI(
    title="Boston Housing Price API - André Victor",
    description="Batch prediction + API Key + modelo embutido (sem download externo)",
    version="4.0"
)

# MODELO EMBUTIDO EM BASE64 (recarrega a cada deploy, mas é instantâneo depois)
MODEL_B64 = """
UEsDBBQAAAAIAH1L0j8f4yC7oAAAAAABAAAAAgAAAAhCRU0gAAAGUAAAAAQAAAAJ6AAG5AAAA
... (base64 completo do modelo - eu vou colar o real abaixo, mas no Colab roda pra gerar)
"""

# Carrega o modelo da string base64
buffer = io.BytesIO(base64.b64decode(MODEL_B64))
model = joblib.load(buffer)

API_KEY = "boston_2025_secret_8f9d2a1c9e7b3f6d5a4c8e2f1d0b9a8e7c6d5f4"

class HouseFeatures(BaseModel):
    CRIM: float = Field(..., example=0.006)
    ZN: float = Field(..., example=18.0)
    INDUS: float = Field(..., example=2.31)
    CHAS: int = Field(..., example=0, ge=0, le=1)
    NOX: float = Field(..., example=0.538)
    RM: float = Field(..., example=6.575)
    AGE: float = Field(..., example=65.2)
    DIS: float = Field(..., example=4.09)
    RAD: int = Field(..., example=1)
    TAX: float = Field(..., example=296.0)
    PTRATIO: float = Field(..., example=15.3)
    B: float = Field(..., example=396.9)
    LSTAT: float = Field(..., example=4.98)

@app.get("/")
def root():
    return {"message": "API do André Victor rodando 24/7 com modelo embutido!"}

@app.post("/predict")
def predict(houses: List[HouseFeatures], x_api_key: str = Header(..., alias="X-API-Key")):
    if x_api_key != API_KEY:
        raise HTTPException(401, "API Key inválida")

    df = pd.DataFrame([h.dict() for h in houses])
    predictions = model.predict(df)

    return {
        "total_predições": len(predictions),
        "preços_em_milhares_de_USD": [round(float(p), 2) for p in predictions]
    }

print("main_final.py criado com sucesso! Copie para o GitHub.")

Writing main_final.py


**HUGGING FACE**

In [ ]:
!pip install huggingface_hub -q

INFO:     Started server process [66021]
INFO:     Started server process [66021]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [ ]:
import joblib
import os
from google.colab import files

# Recarrega o modelo (caso tenha perdido)
# (mesmo código de antes — roda rápido)
import numpy as np, pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# ... (código do modelo sintético — cole o mesmo de antes ou use o que já tem em /tmp)
# Vou assumir que você ainda tem o modelo em /tmp/boston_model.pkl
# Se não tiver, roda isso rápido:

joblib.dump(model, "boston_model.pkl")  # se ainda tiver o objeto 'model'
# ou se perdeu, roda o treinamento de novo (10 segundos)

# Agora faz upload direto pro GitHub como Release (mágico)
!pip install -q requests

import requests

token = input("Cole seu GitHub Personal Access Token (o mesmo de antes): ")
repo = "andrevictorm/boston-housing-api"

# Cria release se não existir
url = f"https://api.github.com/repos/{repo}/releases"
headers = {"Authorization": f"token {token}", "Accept": "application/vnd.github.v3+json"}

# Cria release "model-v1"
data = {"tag_name": "model-v1", "name": "Modelo Boston Housing", "body": "Modelo treinado para API"}
r = requests.post(url, headers=headers, json=data)
if r.status_code in [200, 201]:
    release_id = r.json()["id"]
    print("Release criada!")
else:
    # Se já existe, pega a existente
    r = requests.get(url, headers=headers)
    for release in r.json():
        if release["tag_name"] == "model-v1":
            release_id = release["id"]
            print("Release já existe, usando ela.")
            break

# Faz upload do modelo como asset
upload_url = f"https://uploads.github.com/repos/{repo}/releases/{release_id}/assets?name=boston_model.pkl"
headers = {"Authorization": f"token {token}", "Content-Type": "application/octet-stream"}

with open("boston_model.pkl", "rb") as f:
    r = requests.post(upload_url, headers=headers, data=f)

if r.status_code == 201:
    download_url = r.json()["browser_download_url"]
    print("MODELO SUBIDO COM SUCESSO!")
    print(f"Link direto (100% funcional): {download_url}")
else:
    print("Erro:", r.text)

Release já existe, usando ela.


SSLError: HTTPSConnectionPool(host='uploads.github.com', port=443): Max retries exceeded with url: /repos/andrevictorm/boston-housing-api/releases/264653688/assets?name=boston_model.pkl (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')))

In [ ]:
%%writefile main.py
import joblib
import os
import urllib.request

MODEL_PATH = "boston_model.pkl"
MODEL_URL = "https://github.com/andrevictorm/boston-housing-api/releases/download/model-v1/boston_model.pkl"

if not os.path.exists(MODEL_PATH):
    print("Baixando modelo do GitHub Releases...")
    urllib.request.urlretrieve(MODEL_URL, MODEL_PATH)
    print("Modelo baixado!")

model = joblib.load(MODEL_PATH)

from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
from typing import List
import pandas as pd

app = FastAPI(title="Boston Housing API - André Victor")

API_KEY = "boston_2025_secret_8f9d2a1c9e7b3f6d5a4c8e2f1d0b9a8e7c6d5f4"

class HouseFeatures(BaseModel):
    CRIM: float = Field(..., example=0.00632)
    ZN: float = Field(..., example=18.0)
    INDUS: float = Field(..., example=2.31)
    CHAS: int = Field(..., example=0)
    NOX: float = Field(..., example=0.538)
    RM: float = Field(..., example=6.575)
    AGE: float = Field(..., example=65.2)
    DIS: float = Field(..., example=4.09)
    RAD: int = Field(..., example=1)
    TAX: float = Field(..., example=296.0)
    PTRATIO: float = Field(..., example=15.3)
    B: float = Field(..., example=396.9)
    LSTAT: float = Field(..., example=4.98)

@app.get("/")
def root():
    return {"message": "API do André Victor - funcionando 24/7 com modelo no GitHub Releases"}

@app.post("/predict")
def predict(houses: List[HouseFeatures], x_api_key: str = Header(alias="X-API-Key")):
    if x_api_key != API_KEY:
        raise HTTPException(401, "Chave errada")
    df = pd.DataFrame([h.dict() for h in houses])
    pred = model.predict(df)
    return {"preços (mil USD)": [round(float(p), 2) for p in pred]}

from google.colab import files
files.download('main.py')

Overwriting main.py


OUTRO


In [ ]:
!pip install scikit-learn==1.7.2 --upgrade

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib
from sklearn import __version__
import os

print(f"🔥 Versão ATUALIZADA do scikit-learn: {__version__}")

# Gerar dados sintéticos (mesmo código)
np.random.seed(42)
n_samples = 5000

data = {
    'CRIM': np.random.lognormal(0, 1, n_samples),
    'ZN': np.random.uniform(0, 100, n_samples),
    'INDUS': np.random.uniform(0, 30, n_samples),
    'CHAS': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),
    'NOX': np.random.uniform(0.3, 0.9, n_samples),
    'RM': np.random.normal(6.5, 1, n_samples),
    'AGE': np.random.uniform(1, 100, n_samples),
    'DIS': np.random.lognormal(1, 0.8, n_samples),
    'RAD': np.random.choice(range(1,25), n_samples),
    'TAX': np.random.uniform(200, 800, n_samples),
    'PTRATIO': np.random.uniform(12, 22, n_samples),
    'B': np.random.uniform(0, 400, n_samples),
    'LSTAT': np.random.uniform(1, 40, n_samples),
}

df = pd.DataFrame(data)
df['MEDV'] = (
    10 + 4 * df['RM'] - 0.2 * df['LSTAT'] + 3 * df['CHAS'] -
    2 * np.log1p(df['CRIM']) + 1.5 * (df['ZN']/10) -
    0.05 * df['AGE'] + np.random.normal(0, 3, n_samples)
)
df['MEDV'] = df['MEDV'].clip(lower=5)

# Treinar modelo
X = df.drop('MEDV', axis=1)
y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print(f"🔥 R² no teste: {model.score(X_test, y_test):.4f}")

# Salvar modelo COMPATÍVEL com Railway
joblib.dump(model, 'boston_model_compatible_v2.pkl')
file_size = os.path.getsize('boston_model_compatible_v2.pkl') / (1024*1024)
print(f"🔥 Modelo COMPATÍVEL salvo! Tamanho: {file_size:.2f} MB")
print("🔥 AGORA ESTÁ COMPATÍVEL COM O RAILWAY!")

🔥 Versão ATUALIZADA do scikit-learn: 1.7.2
🔥 R² no teste: 0.7759
🔥 Modelo COMPATÍVEL salvo! Tamanho: 34.72 MB
🔥 AGORA ESTÁ COMPATÍVEL COM O RAILWAY!


In [ ]:
from google.colab import files
files.download('boston_model_compatible_v2.pkl')
print("📥 NOVO modelo compatível baixado! Agora substitua no GitHub.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 NOVO modelo compatível baixado! Agora substitua no GitHub.


**REQUISIÇÃO RENDER**

In [ ]:
import requests

url = "https://boston-housing-api-2.onrender.com/predict"
headers = {
    "X-API-Key": "boston_2025_secret_8f9d2a1c9e7b3f6d5a4c8e2f1d0b9a8e7c6d5f4",
    "Content-Type": "application/json",
    "User-Agent": "MyBostonApp/1.0"
}

def obter_float(mensagem):
    """Função para obter um valor float do usuário com validação"""
    while True:
        try:
            valor = float(input(mensagem))
            return valor
        except ValueError:
            print("❌ Por favor, digite um número válido.")

def obter_int(mensagem):
    """Função para obter um valor inteiro do usuário com validação"""
    while True:
        try:
            valor = int(input(mensagem))
            return valor
        except ValueError:
            print("❌ Por favor, digite um número inteiro válido.")

def cadastrar_casa(numero_casa):
    """Função para cadastrar os dados de uma casa"""
    print(f"\n🏠 CADASTRANDO CASA {numero_casa}")
    print("=" * 40)

    print("\n📊 Por favor, insira os dados da casa:")

    crim = obter_float("CRIM (Taxa de criminalidade per capita): ")
    zn = obter_float("ZN (Proporção de terrenos residenciais zoneados): ")
    indus = obter_float("INDUS (Proporção de acres de negócios não-varejistas): ")
    chas = obter_int("CHAS (Limita com o rio Charles? 1 = Sim, 0 = Não): ")
    nox = obter_float("NOX (Concentração de óxidos nítricos): ")
    rm = obter_float("RM (Número médio de quartos por habitação): ")
    age = obter_float("AGE (Proporção de unidades ocupadas pelo proprietário construídas antes de 1940): ")
    dis = obter_float("DIS (Distância ponderada até cinco centros de emprego de Boston): ")
    rad = obter_int("RAD (Índice de acessibilidade às rodovias radiais): ")
    tax = obter_float("TAX (Taxa de imposto sobre a propriedade de valor total por $10,000): ")
    ptratio = obter_float("PTRATIO (Razão aluno-professor por cidade): ")
    b = obter_float("B (Proporção de pessoas afrodescendentes por cidade): ")
    lstat = obter_float("LSTAT (% de status inferior da população): ")

    # Mostrar resumo para confirmação
    print(f"\n📋 RESUMO DA CASA {numero_casa}:")
    print(f"CRIM: {crim}")
    print(f"ZN: {zn}")
    print(f"INDUS: {indus}")
    print(f"CHAS: {chas}")
    print(f"NOX: {nox}")
    print(f"RM: {rm}")
    print(f"AGE: {age}")
    print(f"DIS: {dis}")
    print(f"RAD: {rad}")
    print(f"TAX: {tax}")
    print(f"PTRATIO: {ptratio}")
    print(f"B: {b}")
    print(f"LSTAT: {lstat}")

    confirmar = input("\n✅ Confirmar estes dados? (s/n): ").lower().strip()

    if confirmar == 's':
        casa = {
            "CRIM": crim, "ZN": zn, "INDUS": indus, "CHAS": chas,
            "NOX": nox, "RM": rm, "AGE": age, "DIS": dis,
            "RAD": rad, "TAX": tax, "PTRATIO": ptratio, "B": b,
            "LSTAT": lstat
        }
        print("✅ Casa cadastrada com sucesso!")
        return casa
    else:
        print("🔄 Reiniciando cadastro desta casa...")
        return cadastrar_casa(numero_casa)

def main():
    print("🏡 SISTEMA DE PREVISÃO DE PREÇOS DE IMÓVEIS - BOSTON")
    print("=" * 55)

    casas = []
    numero_casa = 1

    while True:
        # Cadastrar uma casa
        casa = cadastrar_casa(numero_casa)
        casas.append(casa)
        numero_casa += 1

        # Perguntar se quer adicionar outra casa
        continuar = input("\n📝 Deseja adicionar outra casa? (s/n): ").lower().strip()
        if continuar != 's':
            break

    print(f"\n📨 Enviando dados de {len(casas)} casa(s) para a API...")

    # Fazer a requisição para a API
    response = requests.post(url, json=casas, headers=headers)

    print("Status Code:", response.status_code)

    if response.status_code == 200:
        result = response.json()
        print("\n🎯 PREVISÕES DE PREÇOS DAS CASAS:")
        print("=" * 50)

        for prediction in result["predictions"]:
            print(f"Casa {prediction['house_index'] + 1}: ${prediction['price']:,.2f} mil")

        print("=" * 50)
        print(f"💰 Total de previsões: {len(result['predictions'])} casa(s)")

        # Mostrar estatísticas
        precos = [p['price'] for p in result['predictions']]
        if precos:
            print(f"📈 Preço médio: ${sum(precos)/len(precos):,.2f} mil")
            print(f"🏆 Casa mais cara: ${max(precos):,.2f} mil")
            print(f"💸 Casa mais barata: ${min(precos):,.2f} mil")

    else:
        print(f"❌ Erro na requisição: {response.status_code}")
        print("Resposta da API:")
        print(response.text)

if __name__ == "__main__":
    main()

🏡 SISTEMA DE PREVISÃO DE PREÇOS DE IMÓVEIS - BOSTON

🏠 CADASTRANDO CASA 1

📊 Por favor, insira os dados da casa:
CRIM (Taxa de criminalidade per capita): 20
ZN (Proporção de terrenos residenciais zoneados): 40
INDUS (Proporção de acres de negócios não-varejistas): 11
CHAS (Limita com o rio Charles? 1 = Sim, 0 = Não): 66
NOX (Concentração de óxidos nítricos): 75
RM (Número médio de quartos por habitação): 20
AGE (Proporção de unidades ocupadas pelo proprietário construídas antes de 1940): 3
DIS (Distância ponderada até cinco centros de emprego de Boston): 80
RAD (Índice de acessibilidade às rodovias radiais): 99
TAX (Taxa de imposto sobre a propriedade de valor total por $10,000): 15
PTRATIO (Razão aluno-professor por cidade): 50
B (Proporção de pessoas afrodescendentes por cidade): 70
LSTAT (% de status inferior da população): 45

📋 RESUMO DA CASA 1:
CRIM: 20.0
ZN: 40.0
INDUS: 11.0
CHAS: 66
NOX: 75.0
RM: 20.0
AGE: 3.0
DIS: 80.0
RAD: 99
TAX: 15.0
PTRATIO: 50.0
B: 70.0
LSTAT: 45.0

✅ Con